In [1]:
import numpy as np

In [2]:
#body_xyz_data = np.load('./recording_19_56_30_gmt-4/output_data/mediapipe_body_3d_xyz.npy')
#left_hand_xyz_data = np.load('./recording_19_56_30_gmt-4/output_data/mediapipe_left_hand_3d_xyz.npy')
#right_hand_xyz_data = np.load('./recording_19_56_30_gmt-4/output_data/mediapipe_right_hand_3d_xyz.npy')

In [3]:
# get max distance between left and right hand. this will be the approximated wingspan.

# get difference between max left and max right when left is farthest left and right is farthest right. the one with the greatest distance wins
#left_max = np.max(left_hand_xyz_data[:, :, 0])  # min since 0,0 starts at top-left
#left_max_index = np.where(left_hand_xyz_data == left_max)
#left_max_wingspan = left_hand_xyz_data[left_max_index[0], left_max_index[1], 2][0] - left_max
#print(left_max_wingspan)
#right_max = np.max(right_hand_xyz_data[:, :, 2])  # min since 0,0 starts at top-right
#right_max_index = np.where(right_hand_xyz_data == right_max)
#right_max_wingspan = right_hand_xyz_data[right_max_index[0], right_max_index[1], 0][0] - right_max
#print(right_max_wingspan)

### Taken from example .ipynb

In [4]:
from pathlib import Path

try:
    import numpy as np
except Exception as e:
    print(e)
    %pip install numpy
    import numpy as np


try:
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
except Exception as e:
    print(e)
    %pip install plotly
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    

In [5]:
# function to get data for particular body part
def get_bodypart_data(bodypart = "left_index"):
    path_to_recording = "C:\\Users\\briid\\Documents\\Research\\Arm-Simulation-with-Forces\\experimentation\\freemocap\\recording_19_56_30_gmt-4"
    mediapipe_indices = ['nose',
    'left_eye_inner',
    'left_eye',
    'left_eye_outer',
    'right_eye_inner',
    'right_eye',
    'right_eye_outer',
    'left_ear',
    'right_ear',
    'mouth_left',
    'mouth_right',
    'left_shoulder',
    'right_shoulder',
    'left_elbow',
    'right_elbow',
    'left_wrist',
    'right_wrist',
    'left_pinky',
    'right_pinky',
    'left_index',
    'right_index',
    'left_thumb',
    'right_thumb',
    'left_hip',
    'right_hip',
    'left_knee',
    'right_knee',
    'left_ankle',
    'right_ankle',
    'left_heel',
    'right_heel',
    'left_foot_index',
    'right_foot_index']

    joint_to_plot_index = mediapipe_indices.index(bodypart)


    path_to_recording = Path(path_to_recording)
    path_to_center_of_mass_npy = path_to_recording/'output_data'/'center_of_mass'/'total_body_center_of_mass_xyz.npy'
    path_to_freemocap_3d_body_data_npy = path_to_recording/'output_data'/'mediapipe_body_3d_xyz.npy'

    freemocap_3d_body_data = np.load(path_to_freemocap_3d_body_data_npy)
    total_body_com_data = np.load(path_to_center_of_mass_npy)

    return freemocap_3d_body_data[:,joint_to_plot_index,:]

#### Experimenting:

In [6]:
# get data for both hands
left_hand_data = get_bodypart_data("left_wrist")
right_hand_data = get_bodypart_data("right_wrist")
# and index fingers
left_index_data = get_bodypart_data("left_index")
right_index_data = get_bodypart_data("right_index")

In [7]:
# function to get distance between two body parts
# parameters are the data arrays for the body parts
# returns array of distances between parts
def dist_between_vertices(first_part, second_part):
    cur_dist = np.ndarray(shape = first_part.shape, dtype = first_part.dtype)
    for i in range(0, len(first_part) - 1):     # get the distance for each frame of the video (excluding last frame)
        cur_dist[i] = np.linalg.norm(first_part[i] - second_part[i])
        #print(cur_dist[i])]
    return cur_dist[:-1, 0]

In [8]:
dist = dist_between_vertices(left_index_data, right_index_data)

In [9]:
# find largest distance between both hands
np.max(dist)

921.5067001657222

In [10]:
# get ratio to real distance in meters
real_height_metric = 1.75     # meters
sim_wingspan = np.max(dist)     # max distance between wrists via `freemocap`

sim_to_real_conversion_factor = real_height_metric / sim_wingspan
print(sim_to_real_conversion_factor)

0.0018990637829169152


In this case, we have acquired a sim-to-real (metric) conversion factor of approximately `0.0018990637829`.

For a roughly approximate method of checking the validity of the conversion factor, we will calculate the distance from the eyes (`left_eye` and `right_eye`) to the heels (`left_heel` and `right_heel`).

***Keep in mind, the full body was not in frame for this run at any point, so this estimate is going to be off by a decent margin. This was done for experimentation purposes only.***

In [11]:
# get data for eyes and heels
left_eye_data = get_bodypart_data("left_eye")
right_eye_data = get_bodypart_data("right_eye")
left_heel_data = get_bodypart_data("left_heel")
right_heel_data = get_bodypart_data("right_heel")

In [12]:
# calculate distance for each side, then divide by 2 to get the average of the two
height_approx_left = np.max(dist_between_vertices(left_eye_data, left_heel_data))
height_approx_right = np.max(dist_between_vertices(right_eye_data, right_heel_data))
rough_approx_height_sim = (height_approx_left + height_approx_right) / 2

In [13]:
rough_approx_height = rough_approx_height_sim * sim_to_real_conversion_factor
print("%s meters" % rough_approx_height)

2.3391977451770254 meters


Not great, but only a little less than 25% off from the real value, so not too terrible. Let's try something more reasonable, like the length of one arm.

In [14]:
left_shoulder_data = get_bodypart_data("left_shoulder")
right_shoulder_data = get_bodypart_data("right_shoulder")
np.max(left_shoulder_data)

803.6215345400643

In [15]:
# right hand to shoulder
print("%s meters" % (np.max(dist_between_vertices(right_shoulder_data, right_index_data)) * sim_to_real_conversion_factor))

0.91274777549022 meters


In [16]:
# left hand to shoulder
print("%s meters" % (np.max(dist_between_vertices(left_index_data, left_shoulder_data)) * sim_to_real_conversion_factor))

0.7887509652829577 meters


### Trying to do depth:

- try to do this by calculating the ratio between the current length of any given body part and its longest length.
    - the shorter a part is as compared to its longest length, the more depth needs to be accounted for. 
    - a distance of 0 between the start and end of a body part segment would mean it is directly in line with the camera. a distance of length N where N is the median of the 5 longest measurements of length between two points (i.e. elbow and wrist), should show that the segment is normal to the angle of the camera (thus no depth).
- to figure out which way the depth goes (forwards or back), calculate the difference in light intensity between the vertices in question. whichever one is brighter is closer.
    - could also see whether it's getting brighter or darker over time, which should probably help, I think

In [17]:
# taken from the default output .ipynb
bodypart = 'left_index'
path_to_recording = "C:\\Users\\briid\\Documents\\Research\\Arm-Simulation-with-Forces\\experimentation\\freemocap\\recording_19_56_30_gmt-4"
mediapipe_indices = ['nose',
'left_eye_inner',
'left_eye',
'left_eye_outer',
'right_eye_inner',
'right_eye',
'right_eye_outer',
'left_ear',
'right_ear',
'mouth_left',
'mouth_right',
'left_shoulder',
'right_shoulder',
'left_elbow',
'right_elbow',
'left_wrist',
'right_wrist',
'left_pinky',
'right_pinky',
'left_index',
'right_index',
'left_thumb',
'right_thumb',
'left_hip',
'right_hip',
'left_knee',
'right_knee',
'left_ankle',
'right_ankle',
'left_heel',
'right_heel',
'left_foot_index',
'right_foot_index']

joint_to_plot_index = mediapipe_indices.index(bodypart)


path_to_recording = Path(path_to_recording)
path_to_center_of_mass_npy = path_to_recording/'output_data'/'center_of_mass'/'total_body_center_of_mass_xyz.npy'
path_to_freemocap_3d_body_data_npy = path_to_recording/'output_data'/'mediapipe_body_3d_xyz.npy'

freemocap_3d_body_data = np.load(path_to_freemocap_3d_body_data_npy)
total_body_com_data = np.load(path_to_center_of_mass_npy)

# save pre-modification for testing purposes
body_data_pre_mod = freemocap_3d_body_data

In [18]:
# get indices for body parts
def get_index(body_part = 'left_wrist'):
    return mediapipe_indices.index(body_part)

In [19]:
# function to get data for particular body part
def get_bodypart_data(bodypart = "left_index"):
    
    joint_to_plot_index = mediapipe_indices.index(bodypart)

    return freemocap_3d_body_data[:,joint_to_plot_index,:]

In [20]:
# function to get distance between two body parts
# parameters are the data arrays for the body parts
# returns array of distances between parts
def dist_between_vertices(first_part, second_part):
    cur_dist = np.ndarray(shape = first_part.shape, dtype = first_part.dtype)
    for i in range(0, len(first_part) - 1):     # get the distance for each frame of the video (excluding last frame)
        cur_dist[i] = np.linalg.norm(first_part[i] - second_part[i])
        #print(cur_dist[i])]
    return cur_dist[:-1, 0]

In [21]:
# get body part data
left_index_data = get_bodypart_data("left_index")
left_elbow_data = get_bodypart_data("left_elbow")

In [22]:
# ratio between current length and zero length:
left_hand_to_elbow_array = dist_between_vertices(left_index_data, left_elbow_data)

In [23]:
# get median of largest distances between vertices/bodyparts
def max_dist_between_parts(dist_array):
    ind = (np.argpartition(dist_array, -20)[-20:])
    return np.median(dist_array[ind])

In [24]:
# get approximate distance between vertices/body parts
left_hand_to_elbow_dist = max_dist_between_parts(left_hand_to_elbow_array)
print(left_hand_to_elbow_dist)

285.38804213474384


In [25]:
# calculate the angle of the segment (body part) from the normal (where it is longest)
def angle_from_normal(cur_dist, max_dist):
    return np.arccos(cur_dist / max_dist)

In [26]:
angle_from_normal(0, left_hand_to_elbow_dist)

1.5707963267948966

Nice! The output, `1.570796`, is half of pi, or pi/2. This is exactly what we were looking for when the cur_dist is 0, telling us that it works as expected.

Now, use this to calculate the distance the (body part) vertex is away from the norm with the angle, and there's your depth.

In [27]:
test_vertex_one = get_bodypart_data('left_index')
test_vertex_two = get_bodypart_data('left_elbow')
test_dist_array = dist_between_vertices(test_vertex_one, test_vertex_two)

test_dist_array = test_dist_array
test_max_dist = max_dist_between_parts(test_dist_array)
test_angle = angle_from_normal(test_dist_array[13], test_max_dist)
test_depth = np.sin(test_angle) * test_max_dist
print(test_angle)
print(test_depth)

0.6311801281214334
168.4068910425355


In [28]:
# get depth
#def get_depth(vertex_one, vertex_two, timestep):
#    dist_array = dist_between_vertices(vertex_one, vertex_two)
#    max_dist = max_dist_between_parts(dist_array)
#    cur_dist = dist_array[timestep]
#    angle = angle_from_normal(cur_dist, max_dist)
#    depth = np.sin(angle) * max_dist
#    return depth

In [29]:
# get depth
def get_depth(vertex_one, vertex_two):
    dist_array = dist_between_vertices(vertex_one, vertex_two)
    max_dist = max_dist_between_parts(dist_array)
    depths = list()
    for frame in dist_array:
        angle = angle_from_normal(frame, max_dist)
        depths.append(np.sin(angle) * max_dist)
    return depths

In [30]:
np.nan_to_num(get_depth(get_bodypart_data('left_index'), get_bodypart_data('left_elbow')))#, 10))

C:\Users\briid\AppData\Local\Temp\ipykernel_18344\1837775029.py:3: RuntimeWarning: invalid value encountered in arccos
  return np.arccos(cur_dist / max_dist)


array([104.08532221, 187.42971302, 211.58484526, 205.73885156,
       183.34063811, 160.51295147, 163.38023909, 194.38023133,
       218.77303034, 220.50392945, 204.60231557, 190.94817675,
       186.25539961, 168.40689104, 140.77892016, 180.34991965,
       255.51616282, 281.83438505, 273.87126581, 231.37430981,
       188.02915083, 193.13181951, 200.75048266, 191.1994885 ,
       180.9387394 , 183.60787722, 194.08849059, 202.14630798,
       202.96918812, 196.16554343, 184.91276637, 174.42305759,
       166.16877393, 158.04585151, 150.33612162, 146.82988182,
       149.93156961, 157.54154904, 165.36469616, 170.06955301,
       171.48677369, 172.62364606, 175.71584499, 179.27969096,
       181.27411927, 182.11388664, 182.96267448, 183.88448525,
       184.95604949, 186.79782241, 189.0035991 , 190.79827977,
       193.77898257, 200.29824756, 209.33499963, 217.64896905,
       223.42460241, 226.14586586, 224.90971394, 219.48876605,
       212.74242669, 209.18118589, 209.53842191, 210.42

Test it out by running it for each time step then setting the values for `y` in the data output matrices to the calculated values with `get_depth`. 

Visualize in the same way it does by default in order to check how well it works.

In [31]:
# do depth for each time set, set for each vertex according to its nearest neighbor

In [32]:
# put together pairs for each of the vertices
# the first one is the point which will be moved on the y-axis, the second one for calculations
#vertex_pairs = [    # this set only for upper body
#    ['left_shoulder', 'right_shoulder'],
#    ['left_elbow', 'left_shoulder'],
#    ['left_index', 'left_elbow'],
#    ['right_elbow', 'right_shoulder'],
#    ['right_index','right_elbow']
#]
vertex_order = [
    [
        'left_shoulder',
        'right_shoulder'
    ],
    [
        'left_shoulder',
        'left_elbow',
        'left_wrist',
    ],
    [
        'right_shoulder',
        'right_elbow',
        'right_wrist'
    ]
]

In [33]:
# get length of body parts by vertex pairs
#def get_y_axes(vertex_pairs = vertex_pairs):
#    lengths = list()
#    for vertex_pair in vertex_pairs:
#        lengths.append(np.nan_to_num(get_depth(get_bodypart_data(vertex_pair[0]), get_bodypart_data(vertex_pair[1]))))
#    return lengths

# get y axes by order of body parts
def get_y_axes(vertex_order = vertex_order):
    y = list()
    for vertices in vertex_order:
        group_y = list()
        num_vertices = len(vertices)
        for i, vertex in enumerate(vertices):
            if i < (num_vertices - 1):
                y_dist_between_vertices = np.nan_to_num(get_depth(get_bodypart_data(vertices[i]), get_bodypart_data(vertices[i + 1])))
                if i > 0:
                    vertex_y = group_y[i - 1] +  y_dist_between_vertices    # add y depth of anchor to current
                else:
                    vertex_y = y_dist_between_vertices
                group_y.append(vertex_y)
        y.append(group_y)
    return y



In [34]:
# test get_y_axes
#print(np.asarray(get_y_axes()).shape)
print(get_y_axes())


[[array([143.31928023, 139.80116039, 140.68749802, 146.71933331,
       154.01923789, 158.40285363, 158.64140304, 156.79912863,
       156.6808428 , 161.19099996, 169.39683048, 176.4376383 ,
       178.43639744, 176.22806587, 173.58092897, 172.11881128,
       169.75309186, 165.03543233, 160.08263895, 157.61246684,
       156.93217612, 155.26141184, 151.03124998, 144.6889738 ,
       138.95626598, 138.03581557, 142.57241076, 147.47863764,
       148.28977956, 145.66265915, 142.86057962, 142.11763985,
       145.04052331, 152.45715571, 161.50982654, 167.30501551,
       168.2137883 , 167.27608905, 168.21843334, 171.7366883 ,
       177.29596655, 185.38277077, 195.10544808, 202.93254957,
       206.37309442, 206.64576746, 206.72222694, 207.4276443 ,
       206.94001544, 203.92459304, 199.32063176, 194.84800856,
       190.94276308, 187.43066115, 185.0172412 , 184.18492797,
       184.1512467 , 184.6347519 , 186.45517991, 189.2471586 ,
       191.24048862, 191.97412993, 192.87939278, 194.

C:\Users\briid\AppData\Local\Temp\ipykernel_18344\1837775029.py:3: RuntimeWarning: invalid value encountered in arccos
  return np.arccos(cur_dist / max_dist)


In [35]:
# get y axes
y_axes = get_y_axes()
# account for difference between shoulder y-axes
y_axes[2] += y_axes[0]  # by adding it to the branch off the right shoulder

C:\Users\briid\AppData\Local\Temp\ipykernel_18344\1837775029.py:3: RuntimeWarning: invalid value encountered in arccos
  return np.arccos(cur_dist / max_dist)


In [36]:
# put together dictionary coordinating everything
depth_dict = {
    'vertex_order': vertex_order,                # pairs of body parts/segments
    'y_axes': y_axes,   # approximated depth
}

### Bringing it all together now:

- current setup uses left shoulder as temp anchor

In [37]:
freemocap_3d_body_data[20, :, 1]    # the '1' here indicates the y-axis

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [38]:
# get indices for body parts used
indices = list()
for vertices in depth_dict['vertex_order']:
    for vertex in vertices:
        indices.append(get_index(vertex))

print(indices)

[11, 12, 11, 13, 15, 12, 14, 16]


for all in dict:
    for all vertices:
        set y to y_axis_depth + y_axis_depth(vertex_anchor)     <--vertex_anchor referring to other vertex, i.e. left_shoulder for left_elbow

In [39]:
# approximate depth for the video recording
def set_depth(depth_dict = depth_dict, body_data = freemocap_3d_body_data):
    depth_body_data = body_data[:, :, 1]    # y axis of each part on each frame

    # go through and set y-axes values accordingly
    for i, order_group in enumerate(depth_dict['y_axes']):
        cur_length = len(depth_dict['vertex_order'][i])
        # go thru all vertices in current group
        for j, vertex in enumerate(order_group):
            if j < (cur_length - 1):
                # set y axis for each vertex in the order group
                cur_vertex = depth_dict['vertex_order'][i][j + 1]   # + 1 so that it applies to the non-anchor vertex
                vertex_index = mediapipe_indices.index(cur_vertex)
                body_data[:, vertex_index, 1] = np.append(vertex, 0)

    body_data[:, :, 1] = depth_body_data
    return body_data

In [40]:
# testing...
test_body_data = freemocap_3d_body_data

test_body_data = set_depth(body_data = test_body_data)

In [41]:
print(test_body_data[:, 13, 1])

[133.27142047  48.14946019   0.          95.04564735 133.4943892
 143.17254106 142.67691216 145.0239165  150.97996815 154.37226293
 152.34884085 147.40060603 145.17076656 147.55311564 148.85684208
 142.34660225 125.21238448 107.98024227 108.38709042 116.52406268
 117.14192728 113.10455113 112.00784887 112.34364116 109.56681925
 103.76292509  97.28865443  92.4351586   91.55894634  94.41195946
  97.8184787  101.04921036 106.57716111 114.26687426 120.3588495
 122.35780043 121.2929124  120.48098019 122.09866575 125.36513567
 128.31889106 129.99835506 130.7473881  131.56826784 133.05348858
 134.58400885 135.34932073 135.2968776  134.79644313 134.50980092
 135.70680966 139.3214765  144.43271568 149.14255215 152.58770048
 154.82719925 155.57394281 154.76088394 154.21048256 156.37441559
 160.50620753 163.23372199 162.73729563 160.02906803 157.36116303
 156.11246174 156.12261201 156.62161718 157.10535617 157.44009715
 157.64511526 157.77078803 157.85851263 157.79575314 157.27525303
 156.2168819

In [42]:
# set the depth:
freemocap_3d_body_data = set_depth(body_data = freemocap_3d_body_data)

In [43]:
freemocap_3d_body_data[:, 13, 1]

array([133.27142047,  48.14946019,   0.        ,  95.04564735,
       133.4943892 , 143.17254106, 142.67691216, 145.0239165 ,
       150.97996815, 154.37226293, 152.34884085, 147.40060603,
       145.17076656, 147.55311564, 148.85684208, 142.34660225,
       125.21238448, 107.98024227, 108.38709042, 116.52406268,
       117.14192728, 113.10455113, 112.00784887, 112.34364116,
       109.56681925, 103.76292509,  97.28865443,  92.4351586 ,
        91.55894634,  94.41195946,  97.8184787 , 101.04921036,
       106.57716111, 114.26687426, 120.3588495 , 122.35780043,
       121.2929124 , 120.48098019, 122.09866575, 125.36513567,
       128.31889106, 129.99835506, 130.7473881 , 131.56826784,
       133.05348858, 134.58400885, 135.34932073, 135.2968776 ,
       134.79644313, 134.50980092, 135.70680966, 139.3214765 ,
       144.43271568, 149.14255215, 152.58770048, 154.82719925,
       155.57394281, 154.76088394, 154.21048256, 156.37441559,
       160.50620753, 163.23372199, 162.73729563, 160.02

### Integrating the functions from previous iterations of the project:

In [44]:
# set aside an untouched version of freemocap_3d_body_data for testing purposes
body_data = body_data_pre_mod

#### IMPORTANT: set a bias on all coordinates, so that the left shoulder is [0, 0, 0].

In [45]:
# ONLY RUN THIS ONCE
# # set bias for all frames and body parts so that left shoulder is 0, 0, 0 on the coordinate plane
#for i in range(0, len(freemocap_3d_body_data[0, :, 0])):
#    freemocap_3d_body_data[:, i, :] -= freemocap_3d_body_data[:, 11, :]

#### Getting spherical coordinates from rectangular:

In [46]:
body_data_no_y = freemocap_3d_body_data[:, :, :]
body_data_no_y[:, :, 1] = 0
print(body_data_no_y[:, 13, 1])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [47]:
# starting point
body_data = freemocap_3d_body_data
save_body_data = body_data

In [51]:
body_data = save_body_data

##### Normalization

Getting data with `left_shoulder` as frame of reference (0, 0, 0 on the rectangular coordinate plane)

It seems that the root of the issue is that the coordinates aren't all above/below 0, yet you're still subtracting to get the difference. Normalize first, ya dingus.

Subtract 1000 on the x and y axis, and 500 on the z axis for all of them.

In [ ]:
# get into positive number range for all values
for i in range(0, len(body_data)):
    

In [48]:
# set elbow to be 0, 0, 0 (for calculating spherical coordinates)
# go thru and subtract left_elbow from each of the vertices of interest
for i in range(0, len(body_data)):
    for vertex in [11, 13, 15]:
        # get into positive number range for all values

        # set left_shoulder to be 0, 0, 0
        body_data[i, vertex, :] = body_data[i, vertex, :] - body_data[i, 11, :]

# only run this once

In [57]:
body_data[:, 15, :]

array([[ 752.32367063,    0.        , -796.41154768],
       [ 753.10335668,    0.        , -796.3695673 ],
       [ 747.64563901,    0.        , -786.65744537],
       ...,
       [ 711.60048891,    0.        , -433.09800011],
       [ 716.76220169,    0.        , -432.02822055],
       [ 724.53470815,    0.        , -443.81522639]])

As for the others, one thing we can do is normalize the data between max length and 0 for each of the segments (body parts), as follows:

In [49]:
print(freemocap_3d_body_data[:, 15, 0])

[752.32367063 753.10335668 747.64563901 737.33748539 729.6574583
 726.99345669 720.97578913 704.63978768 686.50876148 681.82972402
 690.71215846 697.21856177 694.52057116 699.52410717 729.4187774
 772.55943362 806.20940005 835.68820835 882.30731173 939.34986084
 977.62782444 988.67309748 989.91416811 993.3694499  994.33770284
 987.43142395 976.63200838 968.93466751 966.8084247  968.37721983
 971.04797322 973.26991328 973.81635078 971.06738109 964.39936611
 955.90251121 948.86215731 944.39230673 941.25538464 938.5125825
 936.11611063 933.64713138 930.85319519 928.59725228 927.48496171
 926.71505456 925.51160231 924.48292298 924.6224102  925.36401762
 923.8569188  916.6594265  902.99685156 885.98630717 869.92752872
 856.93782452 846.59820188 837.77873401 829.92339697 823.11740075
 817.72561019 814.08173046 812.35338939 812.3696178  813.2362804
 813.58000892 812.81707375 811.69443886 810.95649456 810.19656756
 808.89087246 807.84211128 808.32031996 810.09296275 811.61518361
 812.17769768 

In [48]:
# get max length of segments/body parts
upper_arm_max_length = max_dist_between_parts(dist_between_vertices(freemocap_3d_body_data[:, 11, :], freemocap_3d_body_data[:, 13, :]))
lower_arm_max_length = max_dist_between_parts(dist_between_vertices(freemocap_3d_body_data[:, 13, :], freemocap_3d_body_data[:, 15, :]))
print(lower_arm_max_length * sim_to_real_conversion_factor)

0.40339966360352


In [50]:
# x,y,z --> rho,theta,phi
x = body_data[:, 11:17:2, 0]
y = body_data[:, 11:17:2, 1]
z = body_data[:, 11:17:2, 2]

rho = np.sqrt(x **2 + y ** 2 + z ** 2)
theta = np.arctan2(y, x)
phi = np.arccos(z / rho)

In [51]:
body_data[:, 11, :]

array([[ 733.89598884,    0.        , -423.48073111],
       [ 732.31511823,    0.        , -424.27016023],
       [ 728.29903875,    0.        , -425.08994307],
       ...,
       [ 786.91951097,    0.        , -428.31587218],
       [ 793.29990049,    0.        , -430.65171263],
       [ 803.62153454,    0.        , -439.32993694]])

In [62]:
rho[:, 0]

array([847.313, 846.34 , 843.28 , 837.389, 829.272, 820.059, 811.078,
       803.574, 797.489, 791.152, 783.798, 777.703, 775.793, 777.728,
       780.583, 782.66 , 783.961, 783.635, 780.292, 774.953, 770.826,
       769.68 , 770.362, 770.604, 768.499, 762.911, 754.423, 745.545,
       738.696, 734.545, 732.531, 731.379, 729.154, 725.   , 720.958,
       719.977, 722.153, 724.91 , 726.55 , 727.427, 727.912, 727.394,
       725.765, 724.271, 724.025, 724.624, 725.027, 725.113, 725.384,
       725.452, 723.973, 720.417, 715.93 , 711.924, 708.575, 705.019,
       700.098, 692.953, 684.183, 676.364, 671.97 , 670.67 , 669.81 ,
       667.561, 664.588, 662.563, 662.104, 662.609, 663.342, 664.061,
       664.756, 665.292, 665.588, 665.978, 666.869, 667.929, 668.263,
       667.838, 668.136, 670.524, 674.275, 677.307, 678.882, 680.357,
       683.002, 686.259, 688.631, 689.519, 689.633, 689.906, 690.42 ,
       690.437, 689.349, 687.427, 685.53 , 684.144, 683.102, 682.434,
       682.942, 685.

In [63]:
# get spherical coordinates from rectangular coordinates
def get_spher_coord(body_data = freemocap_3d_body_data[:, 13, :]):
    # normalize coords
    body_data -= freemocap_3d_body_data[:, 11, :]   # using left_shoulder as 0, 0, 0/point of origin

    x = body_data[:, 0]
    y = body_data[:, 1]
    z = body_data[:, 2]

    rho = np.sqrt(x **2 + y ** 2 + z ** 2)
    theta = np.arctan2(y, x)
    phi = np.arccos(z / rho)
    return [rho, theta, phi]

In [64]:
# set last few coords to spherical coords for displaying in graph
#freemocap_3d_body_data[:, 30, :] = np.swapaxes(get_spher_coord(freemocap_3d_body_data[:, 11, :]), 0, 1)   # left shoulder
freemocap_3d_body_data[:, 31, :] = np.swapaxes(get_spher_coord(freemocap_3d_body_data[:, 13, :]), 0, 1)   # left elbow
freemocap_3d_body_data[:, 32, :] = np.swapaxes(get_spher_coord(freemocap_3d_body_data[:, 15, :]), 0, 1)   # left wrist


In [55]:
# not scientific notation
np.set_printoptions(suppress = True, precision = 3)

In [56]:

    # add text showing spherical coords -BD
    #text = "Spherical coordinates:\n" +
    #       "\tLeft shoulder: " + str(freemocap_3d_body_data[i, 30, :]) +
    #       "\n\tLeft elbow: " + str(freemocap_3d_body_data[i, 31, :]) +
    #       "\n\tLeft wrist: " + str(freemocap_3d_body_data[i, 32, :])

##### Get angle between upper and lower arm:

In [57]:
# get angle between two segments
#def get_angle_between(bodypart_one, bodypart_two)
#    sim_to_real_conversion_factor

### Looks like it worked! Now onto 3D plotting, using resources again from the default .ipynb generated with the recording from FreeMoCap...

In [58]:
# saving the body data just in case...
save_body_data = freemocap_3d_body_data

In [62]:
freemocap_3d_body_data = save_body_data

In [49]:
freemocap_3d_body_data = body_data

In [50]:
# this cell was copied and pasted straight from the FreeMoCap output .ipynb file

def calculate_axes_means(skeleton_3d_data):
    mx_skel = np.nanmean(skeleton_3d_data[:,0:33,0])
    my_skel = np.nanmean(skeleton_3d_data[:,0:33,1])
    mz_skel = np.nanmean(skeleton_3d_data[:,0:33,2])

    return mx_skel, my_skel, mz_skel

ax_range = 1500

mx_skel, my_skel, mz_skel = calculate_axes_means(freemocap_3d_body_data)

# Create a list of frames
frames = [go.Frame(data=[go.Scatter3d(
    x=freemocap_3d_body_data[i, [11, 13, 15], 0],
    y=freemocap_3d_body_data[i, [11, 13, 15], 1],
    z=freemocap_3d_body_data[i, [11, 13, 15], 2],
    mode='markers',#+text',
    marker=dict(
        size=5,  # Adjust marker size as needed
        color=freemocap_3d_body_data[i, 11:17:2, 1],
        colorscale='Jet',
        opacity=0.8
    )
)], name=str(i),
# add spherical coord annotations -BD
layout = go.Layout(annotations = [
    dict(
        text = "Left shoulder: " + str(freemocap_3d_body_data[i, 30, :]) + 
            # get angle between upper and lower arm
            "Forearm/Upper arm angle: " + str(
                np.arccos(freemocap_3d_body_data[i, 31, :], freemocap_3d_body_data[i, 32, :])
            ),
        x = 0.1, y = 1,
        showarrow = False
    ),
    dict(
        text = "Left elbow: " + str(freemocap_3d_body_data[i, 31, :]),
        x = 0.1, y = 0.9,
        showarrow = False
    ),
    dict(
        text = "Left wrist: " + str(freemocap_3d_body_data[i, 32, :]),
        x = 0.1, y = 0.8,
        showarrow = False
    ),
])
) for i in range(freemocap_3d_body_data.shape[0])]

# Define axis properties
axis = dict(
    showbackground=True,
    backgroundcolor="rgb(230, 230,230)",
    gridcolor="rgb(255, 255, 255)",
    zerolinecolor="rgb(255, 255, 255)",
)

# Create a figure
fig = go.Figure(
    data=[go.Scatter3d(
        x=freemocap_3d_body_data[0, [11, 13, 15], 0],
        y=freemocap_3d_body_data[0, [11, 13, 15], 1],
        z=freemocap_3d_body_data[0, [11, 13, 15], 2],
        mode='markers',
        marker=dict(
            size=5,  # Adjust marker size as needed
            color=freemocap_3d_body_data[0, 11:17:2, 1],
            colorscale='Jet',
            opacity=0.8
        )
    )],
    layout=go.Layout(
        scene=dict(
            xaxis=dict(axis, range = [-500, 1000]),          #range=[mx_skel-ax_range, mx_skel+ax_range]), # Adjust range as needed
            yaxis=dict(axis, range = [-500, 1000]),       #range=[my_skel-ax_range, my_skel+ax_range]), # Adjust range as needed
            zaxis=dict(axis, range = [-1500, 0]),       #range=[mz_skel-ax_range, mz_skel+ax_range]),  # Adjust range as needed
            aspectmode='cube'
        ),
        updatemenus=[dict(
            type='buttons',
            showactive=True,
            buttons=[dict(
                label='Play',
                method='animate',
                args=[None, {"frame": {"duration": 30}}]
            ),
            # add pause button  -BD
            dict(
                args = [[None], {"frame": {"duration": 0, "redraw": False},
                                 "mode": "immediate",
                                 "transition": {"duration": 0}}],
                label = 'Stop',
                method = 'animate'
            )]
        )]
    ),
    frames=frames
)


fig.show()

C:\Users\briid\AppData\Local\Temp\ipykernel_18344\1356368051.py:33: RuntimeWarning: invalid value encountered in arccos
  np.arccos(freemocap_3d_body_data[i, 31, :], freemocap_3d_body_data[i, 32, :])
C:\Users\briid\AppData\Local\Temp\ipykernel_18344\1356368051.py:33: RuntimeWarning: invalid value encountered in arccos
  np.arccos(freemocap_3d_body_data[i, 31, :], freemocap_3d_body_data[i, 32, :])
